In [ ]:
# Define a function to evaluate each speaker's dialogue quality based on the previous criteria
def evaluate_speaker_quality(dataframe):
    # Get unique speakers
    speakers = dataframe['speaker'].unique()
    speaker_scores = []

    # Evaluate each speaker by segmenting their dialogue and applying interactivity scores
    for speaker in speakers:
        # Filter data for each speaker
        speaker_data = dataframe[dataframe['speaker'] == speaker]
        
        # Aggregate scores for each interactivity aspect
        topic_management_scores = []
        tone_appropriateness_scores = []
        opening_scores = []
        closing_scores = []

        # Apply scoring to each segment of the speaker
        for seg_id in speaker_data['segment'].unique():
            segment_texts = speaker_data[speaker_data['segment'] == seg_id]['text'].tolist()
            segment_roles = speaker_data[speaker_data['segment'] == seg_id]['role'].tolist()
            # Score each segment with moderator and multi-party criteria
            scores = score_interactivity_aspects_multiparty(segment_texts, segment_roles)
            topic_management_scores.append(scores[0])
            tone_appropriateness_scores.append(scores[1])
            opening_scores.append(scores[2])
            closing_scores.append(scores[3])
        
        # Calculate average scores for each interactivity aspect for the speaker
        avg_topic_management = sum(topic_management_scores) / len(topic_management_scores) if topic_management_scores else 0
        avg_tone_appropriateness = sum(tone_appropriateness_scores) / len(tone_appropriateness_scores) if tone_appropriateness_scores else 0
        avg_opening = sum(opening_scores) / len(opening_scores) if opening_scores else 0
        avg_closing = sum(closing_scores) / len(closing_scores) if closing_scores else 0
        
        # Append results
        speaker_scores.append((speaker, avg_topic_management, avg_tone_appropriateness, avg_opening, avg_closing))
    
    # Create a DataFrame with the results
    speaker_scores_df = pd.DataFrame(speaker_scores, columns=['Speaker', 'Topic Management', 'Tone Appropriateness', 
                                                              'Conversation Opening', 'Conversation Closing'])
    return speaker_scores_df

# Evaluate speaker quality for the first dialogue
first_dialogue_speaker_scores = evaluate_speaker_quality(data)

# Evaluate speaker quality for the second dialogue
second_dialogue_speaker_scores = evaluate_speaker_quality(data_new)

# Combine the results into one CSV file for easy download
combined_speaker_scores = pd.concat([first_dialogue_speaker_scores.assign(Dialogue="First"),
                                     second_dialogue_speaker_scores.assign(Dialogue="Second")])

# Save to CSV
file_path = '/mnt/data/Speaker_Dialogue_Quality_Scores.csv'
combined_speaker_scores.to_csv(file_path, index=False)

file_path
